# BentoML Example: H2O Classification

This show case considers prostate cancer data and tries to find an algorithm to prognose a certain phase of cancer. The dataset was collected at the Ohio State University Comprehensive Cancer Center and includes demographic and medical data from each of the 380 patients as well as a classifier identifying if the patients tumor has already penetrated the prostatic capsule. This latter event is a clear sign for an advanced cancer state and also helps the doctor to decide on biopsy and treatment methods.

In this show case a deep learning algorithm is used to classify the tumors of the patients into 'penetrating prostatic capsule' and 'not penetrating prostatic capsule'. 

In [2]:
!pip install -I bentoml
!pip install h2o

  Using cached https://files.pythonhosted.org/packages/8c/20/11cdf6bb280894a7af48524b0814688a975b9fd6de9ed22ca99d5f0d3f7a/BentoML-0.2.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/51/bd/23c926cd341ea6b7dd0b2a00aba99ae0f828be89d72b2190f27c11d4b7fb/requests-2.22.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/9a/74/670ae9737d14114753b8c8fdf2e8bd212a05d3b361ab15b44937dfd40985/Flask-1.0.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/9f/57/92a497e38161ce40606c27a86759c6b92dd34fcdb33f64171ec559257c02/Werkzeug-0.15.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5c/4e/ed845436bdf61235401224ced23319c7016d5837c9d1b5dcd2456c082d24/boto3-1.9.164-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/91/93/310fe092039f6b0759a1f8524e9e2c56f8012804fa2a8da4e4289bb74d7c/docker-4.0.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/fc/43/

In [ ]:
import h2o
import bentoml

In [ ]:
h2o.init()

In [ ]:
prostate = h2o.import_file(path="https://raw.githubusercontent.com/multicode/h2o-notebook/master/prostate.csv")
prostate.describe()
type(prostate)

In [ ]:
# import the deep learning estimator module
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
# transform the target variable into a factor
prostate["CAPSULE"] = prostate["CAPSULE"].asfactor()
# construct and define the estimator object 
model = H2ODeepLearningEstimator(activation = "Tanh", hidden = [10, 10, 10], epochs = 100)
# train the model on the whole prostate dataset
model.train(x = list(set(prostate.columns) - set(["ID","CAPSULE"])), y ="CAPSULE", training_frame = prostate)
model.show()

In [ ]:
predictions=model.predict(prostate)
predictions.show()
print(type(model))

# Export with BentoML

In [ ]:
%%writefile h2o_model_service.py
import pandas as pd
import h2o
import bentoml
from bentoml.artifact import H2oModelArtifact
from bentoml.handlers import DataframeHandler

@bentoml.artifacts([H2oModelArtifact('model')])
@bentoml.env(conda_pip_dependencies=['h2o'])
class H2oModelService(bentoml.BentoService):

    @bentoml.api(DataframeHandler)
    def predict(self, df):     
        hf = h2o.H2OFrame(df)
        return self.artifacts.model.predict(hf)

In [ ]:
from h2o_model_service import H2oModelService

# Initialize bentoML model with artifacts

bento_model = H2oModelService.pack(
    model=model
)

# Save bentoML model to directory
saved_path = bento_model.save("/tmp/bento")

# print the directory containing exported model archive (prefixed with model name and version)
print(saved_path)

# Usage scenarios

## Load model from archive

In [ ]:
import bentoml
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/multicode/h2o-notebook/master/prostate.csv")
# Load exported bentoML model archive from path
bento_model = bentoml.load(saved_path)
bento_model.predict(df)

## Install from archive as Pypi package

*For demo purpurse, copy generated model to ./model folder*

In [ ]:
import os
import shutil
shutil.rmtree('./model', ignore_errors=True)
shutil.copytree(saved_path, './model')

In [ ]:
!pip install ./model

In [ ]:
# Your bentoML model class name will become packaged name
import H2oModelService

ms = H2oModelService.load() # call load to ensure all artifacts are loaded
ms.predict(pd.read_csv('https://raw.githubusercontent.com/multicode/h2o-notebook/master/prostate.csv'))

## Use as commandline tool

In [ ]:
!H2oModelService --help

In [ ]:
!H2oModelService info

In [ ]:
!H2oModelService predict --help

In [ ]:
!H2oModelService predict --input https://raw.githubusercontent.com/multicode/h2o-notebook/master/prostate.csv

## Use as REST API server with Docker

#### Build the docker image

In [ ]:
!cd "./model" && docker build -t h2o-model .

#### Run the server with docker image

In [ ]:
!docker run -p 5000:5000 h2o-model

##### Run following curl command to get result from the docker container

```bash
curl -i \
--header "Content-Type: text/csv" \
--request POST \
--data 'ID,CAPSULE,AGE,RACE,DPROS,DCAPS,PSA,VOL,GLEASON\n
1,0,65,1,2,1,1.4,0,6\n
2,0,72,1,3,2,6.7,0,7\n' \
localhost:5000/predict
```